## Database Tools



Database tools are **structured interfaces** that allow applications or agents to **read, write, update, and analyze data** stored in databases.
In LLM and agentic systems, database tools act as **ground-truth connectors**, ensuring responses are based on persisted data instead of model assumptions.

```
LLM / App / Agent  →  Database Tool  →  Database  →  Structured Result
```

---

### Why Database Tools Are Critical

* Provide **authoritative data**
* Enable **state persistence** (memory, history, context)
* Support **analytics and reasoning**
* Allow **transactional actions**
* Reduce hallucinations

---

### Categories of Database Tools

#### Relational Database Tools

* Structured schema
* ACID guarantees
  Examples:
* PostgreSQL
* MySQL

#### NoSQL Database Tools

* Flexible schema
* High scalability
  Examples:
* MongoDB
* Redis

#### Vector Database Tools

* Semantic similarity search
* Embedding-based retrieval
  Examples:
* Chroma
* Pinecone

---

### Database Tool Anatomy

A database tool typically defines:

* Connection details
* Query or operation
* Input parameters
* Output schema
* Error handling

Example:

```
Input : { "ticket_id": "INC001" }
Query : SELECT * FROM tickets WHERE id = ?
Output: { "id": "...", "status": "...", "resolution": "..." }
```

---

###  Relational Database Tool Demonstration (PostgreSQL)

#### Database Schema

```sql
CREATE TABLE tickets (
    id TEXT PRIMARY KEY,
    issue TEXT,
    status TEXT,
    resolution TEXT
);
```

---

#### Database Tool Using FastAPI

```python
import psycopg2
from fastapi import FastAPI

app = FastAPI()

conn = psycopg2.connect(
    dbname="supportdb",
    user="postgres",
    password="password",
    host="localhost"
)

@app.get("/ticket/{ticket_id}")
def get_ticket(ticket_id: str):
    cursor = conn.cursor()
    cursor.execute(
        "SELECT id, issue, status, resolution FROM tickets WHERE id=%s",
        (ticket_id,)
    )
    row = cursor.fetchone()
    return {
        "id": row[0],
        "issue": row[1],
        "status": row[2],
        "resolution": row[3]
    }
```

This API endpoint is a **database tool** backed by PostgreSQL.

---

### Using Database Tools Inside LLM Systems

![Image](https://github.blog/wp-content/uploads/2023/10/LLMapparchitecturediagram.png?fit=4088%2C2148\&utm_source=chatgpt.com)

![Image](https://framerusercontent.com/images/gtzHZpL1UHED29stFrkvMbE9xg.png?height=828\&width=1841\&utm_source=chatgpt.com)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1400/1%2AAR03XQPoBTvosviRB0uAdA.png?utm_source=chatgpt.com)

**Flow**

1. User asks about ticket status
2. LLM selects database tool
3. Tool executes SQL
4. Database returns structured row
5. LLM generates grounded response

---

### Database Tool as a LangChain Tool

```python
from langchain.tools import Tool
import psycopg2

def fetch_ticket(ticket_id: str):
    conn = psycopg2.connect(dbname="supportdb", user="postgres", password="password")
    cur = conn.cursor()
    cur.execute(
        "SELECT issue, status FROM tickets WHERE id=%s",
        (ticket_id,)
    )
    return cur.fetchone()

db_tool = Tool(
    name="TicketDatabaseTool",
    func=fetch_ticket,
    description="Fetch ticket details from PostgreSQL database"
)
```

Now the agent can **query structured data safely**.

---

### Vector Database Tool Demonstration

#### Store Embeddings

```python
from chromadb import Client

client = Client()
collection = client.get_or_create_collection("tickets")

collection.add(
    documents=["VPN not working on Windows"],
    ids=["INC001"]
)
```

#### Query Semantically

```python
collection.query(
    query_texts=["VPN issue"],
    n_results=1
)
```

This database tool enables **semantic retrieval (RAG)**.

---

### Database Tools vs API Tools

| Aspect         | Database Tool | API Tool           |
| -------------- | ------------- | ------------------ |
| Data Source    | Database      | Any system         |
| Query Language | SQL / DSL     | HTTP               |
| Latency        | Low           | Medium             |
| Structure      | Strong        | Flexible           |
| LLM Use        | Memory, RAG   | Actions, retrieval |

---

### Real-World Agentic Use Case

**IT Support Agent**

* Database Tool → Fetch ticket history
* Vector DB Tool → Retrieve similar incidents
* API Tool → Escalate issue
* Cache Tool → Redis for session memory

---

### Production Best Practices

* Read-only DB users for agents
* Query validation / SQL guards
* Connection pooling
* Row-level security
* Async queries for scale